In [2]:
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
imgs = np.load("imgs.npy")

In [4]:
def openTensor(imgStr):
    img = []
    for imgName in imgStr:
        img.append(np.array(Image.open(imgName)))
    return np.array(img)/255

In [5]:
X = tf.placeholder(dtype=tf.float32, shape = ((None, 112, 92, 1)))
Y = tf.placeholder(dtype=tf.float32, shape = ((None, 1)))

In [55]:
Z1 = tf.nn.conv2d(input = X, filter = tf.Variable(initial_value=tf.random_normal(shape = [3, 3, 1, 16])), strides=[1, 2, 2, 1], padding="SAME")
A1 = tf.nn.relu(Z1)

A1 = tf.nn.max_pool(A1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

Z2 = tf.nn.conv2d(input= A1, filter = tf.Variable(initial_value=tf.random_normal(shape = [3, 3, 16, 32])), strides=[1, 2, 2, 1], padding="SAME")
A2 = tf.nn.relu(Z2)

A2 = tf.nn.max_pool(A2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

Z3 = tf.nn.conv2d(input= A2, filter = tf.Variable(initial_value=tf.random_normal(shape = [3, 3, 32, 64])), strides=[1, 2, 2, 1], padding="SAME")
A3 = tf.nn.relu(Z3)

A3 = tf.nn.max_pool(A3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

Z4 = tf.nn.conv2d(input= A3, filter = tf.Variable(initial_value=tf.random_normal(shape = [3, 3, 64, 128])), strides=[1, 2, 2, 1], padding="SAME")
A4 = tf.nn.relu(Z4)

A4 = tf.nn.max_pool(A4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

Z5 = tf.nn.conv2d(input= A4, filter = tf.Variable(initial_value=tf.random_normal(shape = [3, 3, 128, 256])), strides=[1, 2, 2, 1], padding="SAME")
A5 = tf.nn.relu(Z5)

A5 = tf.nn.max_pool(A5, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

In [56]:
F = tf.contrib.layers.flatten(inputs= A5)

In [57]:
Z10 = tf.matmul(F, tf.Variable(initial_value= tf.random_normal(shape = [256, 256])))
A10 = tf.nn.relu(Z10)

Z11 = tf.matmul(A10, tf.Variable(initial_value= tf.random_normal(shape = [256, 200])))
A11 = tf.nn.relu(Z11)

Z12 = tf.matmul(A11, tf.Variable(initial_value= tf.random_normal(shape = [200, 128])))
A12 = tf.nn.relu(Z12)

Z13 = tf.matmul(tf.reshape(tf.concat(values = [A12[0], A12[1]], axis = 0), [1, 256]), tf.Variable(initial_value= tf.random_normal(shape = [256, 1]))) + tf.Variable(initial_value=tf.random_normal(shape = [1, 1]))
A13 = tf.nn.sigmoid(Z13)

In [58]:
def lossFn(Y, y_):
#     return -tf.reduce_mean(Y * (tf.clip_by_value(y_[:, 0], 1e-10,1.0)) + (1 - Y) * (1 - tf.clip_by_value(y_[:, 0], 1e-10,1.0)))
    return tf.reduce_mean(tf.keras.backend.binary_crossentropy(target = Y, output= A13))

In [59]:
LOSS = lossFn(Y, A13)

In [60]:
opt = tf.train.AdamOptimizer(learning_rate= 0.09)
train_step = opt.minimize(LOSS)

In [61]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [62]:
def train(epochs = 100):
    for e in range(epochs):
        loss = 0.0
        for img in imgs[:1000]:
            y = np.array([0]).reshape(-1, 1)
            if img[0][:2] == img[1][:2]:
                y = np.array([1]).reshape(-1, 1)
            _, L = sess.run(fetches = [train_step, LOSS], feed_dict={X: openTensor(img).reshape(-1, 112, 92, 1), Y: y})
            loss += L
        if e % 5 == 0:
            print("epoch {}, loss {}".format(e, loss))

In [63]:
train(200)

epoch 0, loss 3188.476967811589
epoch 5, loss 3188.476967811589


KeyboardInterrupt: 

In [ ]:
l = sess.run(A9, feed_dict={X: openTensor(imgs[0]).reshape(-1, 112, 92, 1)})

In [23]:
l[0].shape

(512,)

In [32]:
A11[0, :]

<tf.Tensor 'strided_slice_10:0' shape=(128,) dtype=float32>